In [68]:
# import models
import mplfinance as mpf
from config import api_key
import requests
import pandas as pd
import time
import datetime
from datetime import time
import matplotlib.pyplot as plt
import cufflinks as cf
import os


In [69]:
#input info used to pass through api 
symbol = input("What ticker do you want to search?")
periodtype = input("Period type (day, month, year)")
frequency = input("Frequency of data")
frequencytype = input("Frequency type (minute, monthly, daily)")
period = input("Period of data")
needExtendedHoursData = input('Extended hours data (True or False)')


What ticker do you want to search?TSMC


KeyboardInterrupt: Interrupted by user

In [49]:
 stocks = ["PEP", "COST", "AVGO", "ADBE", "CSCO", "CMCSA", "TMUS", "INTC", "TXN", "AMD", "QCOM", "AMGN", "HON", "INT", "ADP", "PYPL", "SBUX", "MDLZ", "NFLX", "AMAT", "GILD", "BKNG", "ADI", "CHTR", "ISRG", "VRTX", "REGN", "MU", "CSX", "LRCX", "ATVI", "FISV", "MRNA", "PANW", "KDP", "KLAC", "MNST", "SNPS", "AEP", "FTNT", "KHC", "MAR", "PAYX", "ASML", "CDNS", "EXC", "ORLY", "ADSK", "NXPI", "MRVL", "ABNB", "CRWD", "CTAS", "AZN", "XEL", "MELI", "CTSH", "EA", "LULU", "WBA", "DLTR", "MCHP", "SGEN", "BIDU", "JD", "LCID", "BIIB", "ZM", "DXCM", "ILMN", "DDOG", "IDXX", "WDAY", "FAST", "TEAM", "PCAR", "PDD", "ODFL", "VRSK", "ROST", "CPRT", "EBAY", "SIRI", "ZS", "MTCH", "ANSS", "ALGN", "CEG", "VRSN", "NTES", "SWKS", "SPLK", "OKTA", "DOCU"]

In [62]:
#url for td
url = "https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(symbol)
    
#parameters
key = {'apikey':api_key,
         'periodType':periodtype,
          'frequencyType': frequencytype,
          'frequency': frequency,
          'period': period,
          'needExtendedHoursData': needExtendedHoursData,
         }
#requesting data
content = requests.get(url = url, params=key)

data = content.json()

In [63]:
#create arrays for each category
open_tick = []
high_tick = []
low_tick = []
close_tick = []
volume_tick = []
datetime_tick = []
#iterrates through data to append each array with it's value
for i in range(len(data["candles"])):
    open_tick.append(data["candles"][i]["open"])
    high_tick.append(data["candles"][i]["high"])
    low_tick.append(data["candles"][i]["low"])
    close_tick.append(data["candles"][i]["close"])
    volume_tick.append(data["candles"][i]["volume"])
    datetime_tick.append(data["candles"][i]["datetime"])

#converting epoch to datetime       
file = []
for i in range(len(datetime_tick)):
    # divide epoch by 1000 because it is in milliseconds
    test = datetime_tick[i]/1000
    file.append(datetime.datetime.fromtimestamp(test))

#convert to dataframe
theta_df = pd.DataFrame({
            "Open":open_tick,
            "Close":close_tick,
            "High":high_tick,
            "Low":low_tick,
            "Volume":volume_tick,
            "Datetime": file
    })

In [64]:
# set index to Datetime
theta_df = theta_df.set_index("Datetime")

In [65]:
# Calculate % gain for the day 
theta_df["Gain%"] = (theta_df.Close - theta_df.Open)/theta_df.Open * 100

theta_df

,Open,Close,High,Low,Volume,Gain%
Datetime,,,,,,
2002-06-30 22:00:00,24.575,21.990,25.065,19.64,188533400,-10.518820
2002-07-31 22:00:00,21.875,22.315,23.900,19.43,128710400,2.011429
2002-08-31 22:00:00,22.160,22.765,24.475,21.57,139359200,2.730144
2002-09-30 22:00:00,22.805,25.960,26.740,21.50,167557200,13.834685
2002-10-31 22:00:00,25.980,26.970,27.520,24.18,151581800,3.810624
...,...,...,...,...,...,...
2022-01-31 22:00:00,556.400,474.370,579.960,450.26,33932863,-14.742991
2022-02-28 22:00:00,476.130,480.840,492.870,423.03,48114157,0.989226
2022-03-31 22:00:00,480.640,418.750,507.710,416.16,33249299,-12.876581


In [66]:
# for the stock ticker if a folder does not exist already, one will be created
mypath = f"{symbol}"
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [67]:
# uploads csv to that folder specifying the frequency and type to identify the csv
upload_file = f"{symbol}/{symbol}_{frequencytype}_{frequency}_{needExtendedHoursData}.csv"
theta_df.to_csv(upload_file, index = True)

In [70]:
stocks = ["TCEHY", "TSM", "BABA", "ASML", "AVGO", "ORCL", "CSCO", "ADBE", "CRM", "INTC", "QCOM", "TXN", "IBM", "AMD", "SAP", "INTU", "SONY", "JD", "NOW", "ADP", "PYPL", "AMAT", "PDD", "NFLX", "ADI", "BKNG", "MU", "ATVI", "ABNB", "FIS", "FISV", "BIDU"]

In [71]:

periodtype = "year"
frequency = "1"
frequencytype = "monthly"
period = "20"
needExtendedHoursData = "False"

In [72]:
for symbol in stocks:
    #url for td
    url = "https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(symbol)

    #parameters
    key = {'apikey':api_key,
             'periodType':periodtype,
              'frequencyType': frequencytype,
              'frequency': frequency,
              'period': period,
              'needExtendedHoursData': needExtendedHoursData,
             }
    #requesting data
    content = requests.get(url = url, params=key)

    data = content.json()

    #create arrays for each category
    open_tick = []
    high_tick = []
    low_tick = []
    close_tick = []
    volume_tick = []
    datetime_tick = []
    #iterrates through data to append each array with it's value
    for i in range(len(data["candles"])):
        open_tick.append(data["candles"][i]["open"])
        high_tick.append(data["candles"][i]["high"])
        low_tick.append(data["candles"][i]["low"])
        close_tick.append(data["candles"][i]["close"])
        volume_tick.append(data["candles"][i]["volume"])
        datetime_tick.append(data["candles"][i]["datetime"])

    #converting epoch to datetime       
    file = []
    for i in range(len(datetime_tick)):
        # divide epoch by 1000 because it is in milliseconds
        test = datetime_tick[i]/1000
        file.append(datetime.datetime.fromtimestamp(test))

    #convert to dataframe
    theta_df = pd.DataFrame({
                "Open":open_tick,
                "Close":close_tick,
                "High":high_tick,
                "Low":low_tick,
                "Volume":volume_tick,
                "Datetime": file
        })
    # set index to Datetime
    theta_df = theta_df.set_index("Datetime")

    # Calculate % gain for the day 
    theta_df["Gain%"] = (theta_df.Close - theta_df.Open)/theta_df.Open * 100

    # for the stock ticker if a folder does not exist already, one will be created
    mypath = f"{symbol}"
    if not os.path.isdir(mypath):
       os.makedirs(mypath)

    # uploads csv to that folder specifying the frequency and type to identify the csv
    upload_file = f"{symbol}/{symbol}_{frequencytype}_{frequency}_{needExtendedHoursData}.csv"
    theta_df.to_csv(upload_file, index = True)